# **Modeling Global Surface Temperature Data**
This notebook will create a new BigQuery dataset to begin improving the usability of the data. New tables will be created 

In [1]:
!bq --location=US mk --dataset kaggle_modeled

Dataset 'electric-spark-266716:kaggle_modeled' successfully created.


#### **The following queries creates the new modeled tables; the Global_Land_Temperatures_by_City table and Global_Land_Temperatures_by_Major_City table were combined into the City entity table**

In [11]:
%%bigquery
CREATE TABLE kaggle_modeled.Country as
SELECT DISTINCT dt, AverageTemperature,AverageTemperatureUncertainty,Country
FROM kaggle_staging.Global_Land_Temperatures_by_Country
WHERE AverageTemperature is NOT NULL

""


In [13]:
%%bigquery
CREATE TABLE kaggle_modeled.State as
SELECT DISTINCT dt,AverageTemperature,AverageTemperatureUncertainty,
State,Country
FROM kaggle_staging.Global_Land_Temperatures_by_State
WHERE AverageTemperature is NOT NULL

""


In [19]:
%%bigquery
CREATE TABLE kaggle_modeled.Date as
SELECT DISTINCT dt, LandAverageTemperature,LandAverageTemperatureUncertainty,
cast(LandMaxTemperature as NUMERIC) as LandMaxTemperature, 
cast(LandMaxTemperatureUncertainty as NUMERIC) as LandMaxTemperatureUncertainty,
cast(LandMinTemperature as NUMERIC) as LandMinTemperature,
cast(LandMinTemperatureUncertainty as NUMERIC) as LandMinTemperatureUncertainty,
cast(LandAndOceanAverageTemperature as NUMERIC) as LandAndOceanAverageTemperature,
cast(LandAndOceanAverageTemperatureUncertainty as NUMERIC) as LandAndOceanAverageTemperatureUncertainty
FROM kaggle_staging.Global_Temperatures
WHERE LandAverageTemperature is NOT NULL

""


In [20]:
%%bigquery
CREATE TABLE kaggle_modeled.City as
Select DISTINCT Global_Land_Temperatures_by_City.dt,AverageTemperature,
AverageTemperatureUncertainty, Global_Land_Temperatures_by_City.City,Country,Latitude,
Longitude,major_city
FROM kaggle_staging.Global_Land_Temperatures_by_City
FULL JOIN (SELECT City, 1 as major_city,dt
FROM kaggle_staging.Global_Land_Temperatures_by_Major_City) as y
ON Global_Land_Temperatures_by_City.City = y.City and Global_Land_Temperatures_by_City.dt = y.dt
WHERE AverageTemperature is NOT NULL

""


## **Checking Primary Keys**
#### After creating the tables, we ran the following queries to validate the anticipated primary keys for the modeled tables

#### **dt Attribute is a valid primary key for Date table**

In [37]:
%%bigquery
SELECT COUNT(*) as Date_Record_Count,COUNT(DISTINCT dt) as Distinct_Dates
FROM kaggle_modeled.Date

,Date_Record_Count,Distinct_Dates
0,3180,3180


#### **dt, City is not valid primary key for City table**

In [38]:
%%bigquery 
SELECT COUNT(*) as City_Record_Count
FROM kaggle_modeled.City

,City_Record_Count
0,8235082


In [39]:
%%bigquery 
SELECT COUNT(*) as City_Unique_Count
FROM (SELECT DISTINCT dt,y.City
FROM kaggle_modeled.City as y)

,City_Unique_Count
0,8096342


#### **dt, Country is valid primary key for Country table**

In [40]:
%%bigquery 
SELECT COUNT(*) as Country_Record_Count
FROM kaggle_modeled.Country

,Country_Record_Count
0,544811


In [41]:
%%bigquery 
SELECT COUNT(*) as Country_Unique_Count
FROM (SELECT DISTINCT dt,y.Country
FROM kaggle_modeled.Country as y)

,Country_Unique_Count
0,544811


#### **dt, State is valid primary key for State table**

In [42]:
%%bigquery 
SELECT COUNT(*) as State_Record_Count
FROM kaggle_modeled.State

,State_Record_Count
0,620027


In [43]:
%%bigquery 
SELECT COUNT(*) as State_Unique_Count
FROM (SELECT DISTINCT dt,y.State
FROM kaggle_modeled.State as y)

,State_Unique_Count
0,620027


## **Checking Foreign Keys**
The following queries were ran to see if there are any foreign key violations, with the dt attribute of City, Country, and State used as foreign keys to the dt attribute of the Date entity. 

In [25]:
%%bigquery 
SELECT COUNT(*) as City_Unmatched_Elements
FROM kaggle_modeled.City
LEFT JOIN kaggle_modeled.Date
ON City.dt = Date.dt
WHERE Date.dt IS NULL

,City_Unmatched_Elements
0,9855


In [46]:
%%bigquery 
SELECT COUNT(*) as Country_Unmatched_Elements
FROM kaggle_modeled.Country
LEFT JOIN kaggle_modeled.Date
ON Country.dt = Date.dt
WHERE Date.dt IS NULL

,Country_Unmatched_Elements
0,700


In [47]:
%%bigquery 
SELECT COUNT(*) as State_Unmatched_Elements
FROM kaggle_modeled.State
LEFT JOIN kaggle_modeled.Date
ON State.dt = Date.dt
WHERE Date.dt IS NULL

,State_Unmatched_Elements
0,1110


#### The Country attribute of the State entity table is a valid foreign key; has a many-to-many relationship with Country.Country

In [54]:
%%bigquery 
SELECT COUNT(*) as State_Unmatched_Elements
FROM kaggle_modeled.State
LEFT JOIN kaggle_modeled.Country
ON State.Country = Country.Country
WHERE Country.Country IS NULL

,State_Unmatched_Elements
0,0


#### The Country attribute of the City entity table is a valid foreign key; has a many-to-many relationship with Country.Country

In [55]:
%%bigquery 
SELECT COUNT(*) as City_Unmatched_Elements
FROM kaggle_modeled.City
LEFT JOIN kaggle_modeled.Country
ON City.Country = Country.Country
WHERE Country.Country IS NULL

,City_Unmatched_Elements
0,0


## **Beam Pipelines**
The following scripts execute the beam pipelines created.

#### **Pipelines for City Table**

In [1]:
%run City_beam.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []


In [4]:
%run City_beam_dataflow.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:740: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))


#### **Pipelines for Country Table**

In [3]:
%run Country_beam.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []


In [5]:
%run Country_beam_dataflow.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:740: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))


#### **Pipelines for State Table**

In [12]:
%run State_beam.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/io/gcp/bigquery.py:1421: BeamDeprecationWarning: options is deprecated since First stable release. References to <pipeline>.options will not be supported
  experiments = p.options.view_as(DebugOptions).experiments or []


In [13]:
%run State_beam_dataflow.py

/home/jupyter/venv/lib/python3.5/site-packages/apache_beam/runners/dataflow/dataflow_runner.py:740: BeamDeprecationWarning: BigQuerySink is deprecated since 2.11.0. Use WriteToBigQuery instead.
  kms_key=transform.kms_key))


## **Beam Verification**
The following scripts verify if the resulting beam tables have primary and foreign keys. 

#### **City_Beam_DF**
This table has a primary key of dt,City and foreign keys of Country and dt, as shown by the following queries.

In [8]:
%%bigquery 
SELECT COUNT(*) as City_Beam_DF_Record_Count
FROM kaggle_modeled.City_Beam_DF

,City_Beam_DF_Record_Count
0,8046594


In [9]:
%%bigquery 
SELECT COUNT(*) as City_Beam_DF_Unique_Count
FROM (SELECT DISTINCT dt,y.City
FROM kaggle_modeled.City_Beam_DF as y)

,City_Beam_DF_Unique_Count
0,8046594


In [14]:
%%bigquery 
SELECT COUNT(*) as City_Beam_DF_Unmatched_Country_Elements
FROM kaggle_modeled.City_Beam_DF
LEFT JOIN kaggle_modeled.Country
ON City_Beam_DF.Country = Country.Country
WHERE Country.Country IS NULL

,City_Beam_DF_Unmatched_Country_Elements
0,0


In [16]:
%%bigquery 
SELECT COUNT(*) as City_Beam_DF_Unmatched_dt_Elements
FROM kaggle_modeled.City_Beam_DF
LEFT JOIN kaggle_modeled.Date
ON City_Beam_DF.dt = Date.dt
WHERE Date.dt IS NULL

,City_Beam_DF_Unmatched_dt_Elements
0,0


#### **City_Beam**
This table has a primary key of dt,City and foreign keys of Country and dt, as shown by the following queries.

In [2]:
%%bigquery 
SELECT COUNT(*) as City_Beam_Record_Count
FROM kaggle_modeled.City_Beam

,City_Beam_Record_Count
0,20


In [3]:
%%bigquery 
SELECT COUNT(*) as City_Beam_Unique_Count
FROM (SELECT DISTINCT dt,y.City
FROM kaggle_modeled.City_Beam as y)

,City_Beam_Unique_Count
0,20


In [6]:
%%bigquery 
SELECT COUNT(*) as City_Beam_Unmatched_Country_Elements
FROM kaggle_modeled.City_Beam
LEFT JOIN kaggle_modeled.Country
ON City_Beam.Country = Country.Country
WHERE Country.Country IS NULL

,City_Beam_Unmatched_Country_Elements
0,0


In [7]:
%%bigquery 
SELECT COUNT(*) as City_Beam_Unmatched_dt_Elements
FROM kaggle_modeled.City_Beam
LEFT JOIN kaggle_modeled.Date
ON City_Beam.dt = Date.dt
WHERE Date.dt IS NULL

,City_Beam_Unmatched_dt_Elements
0,0


#### **Country_Beam_DF**
This table has a primary key of dt,Country and a foreign key of dt, as shown by the following queries.

In [6]:
%%bigquery 
SELECT COUNT(*) as Country_Beam_DF_Record_Count
FROM kaggle_modeled.Country_Beam_DF

,Country_Beam_DF_Record_Count
0,541167


In [7]:
%%bigquery 
SELECT COUNT(*) as Country_Beam_DF_Unique_Count
FROM (SELECT DISTINCT dt,y.Country
FROM kaggle_modeled.Country_Beam_DF as y)

,Country_Beam_DF_Unique_Count
0,541167


In [8]:
%%bigquery 
SELECT COUNT(*) as Country_Beam_DF_Unmatched_dt_Elements
FROM kaggle_modeled.Country_Beam_DF
LEFT JOIN kaggle_modeled.Date
ON Country_Beam_DF.dt = Date.dt
WHERE Date.dt IS NULL

,Country_Beam_DF_Unmatched_dt_Elements
0,0


#### **Country_Beam**
This table has a primary key of dt,Country and a foreign key of dt, as shown by the following queries.

In [9]:
%%bigquery 
SELECT COUNT(*) as Country_Beam_Record_Count
FROM kaggle_modeled.Country_Beam

,Country_Beam_Record_Count
0,19


In [10]:
%%bigquery 
SELECT COUNT(*) as Country_Beam_Unique_Count
FROM (SELECT DISTINCT dt,y.Country
FROM kaggle_modeled.Country_Beam as y)

,Country_Beam_Unique_Count
0,19


In [11]:
%%bigquery 
SELECT COUNT(*) as Country_Beam_Unmatched_dt_Elements
FROM kaggle_modeled.Country_Beam
LEFT JOIN kaggle_modeled.Date
ON Country_Beam.dt = Date.dt
WHERE Date.dt IS NULL

,Country_Beam_Unmatched_dt_Elements
0,0


#### **State_Beam_DF**
This table has a primary key of dt,State and foreign keys of dt and Country, as shown by the following queries.

In [14]:
%%bigquery 
SELECT COUNT(*) as State_Beam_DF_Record_Count
FROM kaggle_modeled.State_Beam_DF

,State_Beam_DF_Record_Count
0,614707


In [15]:
%%bigquery 
SELECT COUNT(*) as State_Beam_DF_Unique_Count
FROM (SELECT DISTINCT dt,y.State
FROM kaggle_modeled.State_Beam_DF as y)

,State_Beam_DF_Unique_Count
0,614707


In [16]:
%%bigquery 
SELECT COUNT(*) as State_Beam_DF_Unmatched_dt_Elements
FROM kaggle_modeled.State_Beam_DF
LEFT JOIN kaggle_modeled.Date
ON State_Beam_DF.dt = Date.dt
WHERE Date.dt IS NULL

,State_Beam_DF_Unmatched_dt_Elements
0,0


In [17]:
%%bigquery 
SELECT COUNT(*) as State_Beam_DF_Unmatched_Country_Elements
FROM kaggle_modeled.State_Beam_DF
LEFT JOIN kaggle_modeled.Country
ON State_Beam_DF.Country = Country.Country
WHERE Country.Country IS NULL

,State_Beam_DF_Unmatched_Country_Elements
0,0


#### **State_Beam**
This table has a primary key of dt,State and foreign keys of dt and Country, as shown by the following queries.

In [18]:
%%bigquery 
SELECT COUNT(*) as State_Beam_Record_Count
FROM kaggle_modeled.State_Beam

,State_Beam_Record_Count
0,20


In [19]:
%%bigquery 
SELECT COUNT(*) as State_Beam_Unique_Count
FROM (SELECT DISTINCT dt,y.State
FROM kaggle_modeled.State_Beam as y)

,State_Beam_Unique_Count
0,20


In [20]:
%%bigquery 
SELECT COUNT(*) as State_Beam_Unmatched_dt_Elements
FROM kaggle_modeled.State_Beam
LEFT JOIN kaggle_modeled.Date
ON State_Beam.dt = Date.dt
WHERE Date.dt IS NULL

,State_Beam_Unmatched_dt_Elements
0,0


In [21]:
%%bigquery 
SELECT COUNT(*) as State_Beam_Unmatched_Country_Elements
FROM kaggle_modeled.State_Beam
LEFT JOIN kaggle_modeled.Country
ON State_Beam.Country = Country.Country
WHERE Country.Country IS NULL

,State_Beam_Unmatched_Country_Elements
0,0
